In [1]:
import requests
import polars as pl
from tqdm import tqdm
import time

url = 'https://gpnbonus.ru/api/stations/list'
# Заголовки для запроса
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Получение данных
response = requests.get(url, headers=headers)
# Проверка статуса ответа
if response.status_code == 200:
    try:
        data = response.json()  # Декодируем JSON
        print("Данные успешно получены.")
        
        # Сохранение данных в файл
        with open('data.json', 'w', encoding='utf-8') as f:
            import json
            json.dump(data, f, ensure_ascii=False, indent=4)  # Сохраняем в формате JSON
        print("Данные успешно сохранены в 'data.json'.")
        
    except ValueError as e:
        print("Ошибка декодирования JSON:", e)
        print("Ответ сервера:", response.text)  # Вывод текста ответа для диагностики
else:
    print(f"Ошибка при запросе: {response.status_code}")
    print("Ответ сервера:", response.text)  # Вывод текста ответа для диагностики


Данные успешно получены.
Данные успешно сохранены в 'data.json'.


In [2]:
df_promo = pl.DataFrame(data['promotions'])
df_promo.head(1)

id,can_register,conditions,description,details,finishLoc,imageUrl,showMainPage,mainImageUrl,mobileImageUrl,mobile_url,sort,startLoc,title,topImageUrl,unpublished,updated_at,url,regions,excludeMode,excludeRegionMode
i64,bool,str,str,str,str,str,bool,str,str,null,i64,str,str,str,str,str,null,list[i64],i64,i64
850,false,"""<p>С 17 марта по 31 декабря 20…","""Получайте акционные бонусы за …","""<p><strong>Для начисления акци…","""31 декабря 2025""","""/storage/promotions/Ec3ljlwSv7…",true,"""/storage/promotions/A4Blt8fTli…","""/storage/promotions/0OOkWKCevk…",null,100,"""17 марта 2025""","""КОФЕ ЗАПРАВЛЯЕТ""","""/storage/promotions/Grk6lPTiBn…","""2025-12-31T20:59:59.000000Z""","""2025-03-13T12:19:09.000000Z""",null,[],0,0


In [3]:
stations_data = data['stations']
    
    # Извлечение информации о топливе
azs_data = []
for station in stations_data:
    for fuel in station['oils']:
        azs_data.append({
            'GPNAZSID': station['GPNAZSID'],
            'name': station['name'],
            'city': station['city'],
            'address': station['address'],
            'latitude': station['latitude'],
            'longitude': station['longitude'],
            'promotions': station['promotions'],
            'wash': station['wash'],
            'open': station['open'],
            'oils': fuel
        })  

In [4]:
df_base = pl.DataFrame(azs_data)
df_base.head(1)

GPNAZSID,name,city,address,latitude,longitude,promotions,wash,open,oils
i64,str,str,str,str,str,list[i64],bool,bool,i64
3002,"""АЗС № 138 ""","""Новосибирск""","""Покатная, 124""","""55.02546""","""82.94943""","[873, 874]",false,true,12


In [5]:
station_ids = data['stations']
    
    # Извлечение информации о топливе
ids_data = []
for station in station_ids:
    ids_data.append({
        'GPNAZSID': station['GPNAZSID']
    })

In [6]:
df_ids = pl.DataFrame(ids_data)
df_ids.head(1)


GPNAZSID
i64
3002


In [7]:
list_azs = list(df_ids['GPNAZSID'])
len(list_azs)

1846

In [8]:
# df_fuel_agg_base = None

# default_product = {
#     'emisId': None,
#     'color': 'N/A',
#     'shortTitle': 'N/A',
#     'title': 'Нет данных',
#     'type': 'N/A'
# }

# default_price = {
#     'currency': 'RUB',
#     'price': 0.0,
#     'since': 'N/A'
# }

# for i, elem in tqdm(enumerate(list_azs)):
#     print(elem)
#     url = f'https://gpnbonus.ru/api/stations/{elem}'
#     # Заголовки для запроса
#     headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
#     }
    
#     # Получение данных
#     response_temp = requests.get(url, headers=headers)
#     data_temp = response_temp.json() 
    
#     fuel_data = data_temp['data']
#     df_fuel = pl.DataFrame(fuel_data)    
#     df_fuel_agg = df_fuel.with_columns([
#         pl.lit(f'{elem}').cast(pl.Int64).alias('GPNAZSID'),
#         pl.col("product").struct.field("emisId").alias("product_emisId"),
#         pl.col("product").struct.field("color").alias("product_color"),
#         pl.col("product").struct.field("shortTitle").alias("product_shortTitle"),
#         pl.col("product").struct.field("title").alias("product_title"),
#         pl.col("product").struct.field("type").alias("product_type"),
#         pl.col("price").struct.field("currency").alias("price_currency"),
#         pl.col("price").struct.field("price").cast(pl.Float64).alias("price_value"),
#         pl.col("price").struct.field("since").alias("price_since")
#     ])
#     df_fuel_agg = df_fuel_agg.drop(['product', 'price'])

#     if df_fuel_agg_base is None:
#         df_fuel_agg_base = df_fuel_agg
#     else:
#         df_fuel_agg_base = pl.concat([df_fuel_agg_base, df_fuel_agg], how='vertical')
    
#     itog = df_base.join(df_fuel_agg_base.rename({"id": "oils"}), on = ['GPNAZSID', 'oils'], how='left')
#     time.sleep(1)


In [9]:
df_fuel_agg_base = None

default_product = {
    'emisId': None,
    'color': 'N/A',
    'shortTitle': 'N/A',
    'title': 'Нет данных',
    'type': 'N/A'
}

default_price = {
    'currency': 'RUB',
    'price': 0.0,
    'since': 'N/A'
}

for i, elem in tqdm(enumerate(list_azs)):
    print(elem)
    url = f'https://gpnbonus.ru/api/stations/{elem}'
    # Заголовки для запроса
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    # Получение данных
    response_temp = requests.get(url, headers=headers)
    data_temp = response_temp.json() 
    
    processed_data = []

    
    for item in data_temp['data']:
        product = item.get('product', default_product)
        price = item.get('price', default_price)
            
            # Если price пустой, заменяем на значение по умолчанию
        if not price:
            price = default_price
            
        processed_data.append({
            'id': item.get('id'),
            'product': product,
            'price': price
        })
    
    df_fuel = pl.DataFrame(processed_data) 
    df_fuel_agg = df_fuel.with_columns([
        pl.lit(f'{elem}').cast(pl.Int64).alias('GPNAZSID'),
        pl.col("product").struct.field("emisId").alias("product_emisId"),
        pl.col("product").struct.field("color").alias("product_color"),
        pl.col("product").struct.field("shortTitle").alias("product_shortTitle"),
        pl.col("product").struct.field("title").alias("product_title"),
        pl.col("product").struct.field("type").alias("product_type"),
        pl.col("price").struct.field("currency").alias("price_currency"),
        pl.col("price").struct.field("price").cast(pl.Float64).alias("price_value"),
        pl.col("price").struct.field("since").alias("price_since")
    ])
    df_fuel_agg = df_fuel_agg.drop(['product', 'price'])

    if df_fuel_agg_base is None:
        df_fuel_agg_base = df_fuel_agg
    else:
        df_fuel_agg_base = pl.concat([df_fuel_agg_base, df_fuel_agg], how='vertical')
    
    itog = df_base.join(df_fuel_agg_base.rename({"id": "oils"}), on = ['GPNAZSID', 'oils'], how='left')
    time.sleep(1)


0it [00:00, ?it/s]

3002


1it [00:01,  1.36s/it]

1261


2it [00:02,  1.34s/it]

...

1844it [42:08,  1.39s/it]

400000358


1845it [42:09,  1.38s/it]

400000823


1846it [42:11,  1.37s/it]


In [10]:
itog.sort('GPNAZSID').write_excel('1.xlsx')